In [1]:
from functools import partial
import mysql.connector
import re

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        # print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)

In [2]:
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='nlp',
    database='nlp',
)
db = partial(run_db_query, connection)

def word_filter(word: str):
    return re.match("[\s+|0-9]+", word) is None

def word_map(word: str):
    return word.lower()

def extract_word_set(content: str):
    wordlist_dirty = re.sub("\.", "", content)
    wordlist_dirty = re.sub("(|)", "", wordlist_dirty)
    wordlist_dirty = re.sub("([^\x00-\x7F])+"," ",wordlist_dirty)
    wordlist_dirty = re.split("\s+",wordlist_dirty)
    return set(map(word_map,filter(word_filter,wordlist_dirty)))

inv_list = {}
content_list = {}
with connection.cursor() as cursor:
    cursor.execute("SELECT content, id FROM Pages", None)
    for result in cursor:
        content_list[result[1]] = result[0]
        word_set = extract_word_set(result[0])
        for word in word_set:
            if word in inv_list:
                inv_list[word].append(result[1])
            else:
                inv_list[word] = [result[1]]
    print(len(inv_list.keys()))
del inv_list[list(inv_list.keys())[0]]

18713


In [5]:
db("SELECT word FROM Words LIMIT 10")

('(aka',)
('scrapemark',)
('come',)
('our',)
('supposed',)
('into',)
('starting',)
('crawler,',)
('try',)
('user',)


In [3]:
db("SELECT * FROM Words LIMIT 10")

(1, '(aka')
(2, 'scrapemark')
(3, 'come')
(4, 'our')
(5, 'supposed')
(6, 'into')
(7, 'starting')
(8, 'crawler,')
(9, 'try')
(10, 'user')


In [ ]:
db("DELETE FROM Words")
for word in inv_list.keys():
    db(f"INSERT INTO Words (word) VALUES (\"{word}\")")

In [4]:
db("SELECT id FROM Words WHERE word = 'scrapemark'")

(2,)


In [5]:
word_id = {}
all_words = list(inv_list.keys())
with connection.cursor() as cursor:
    cursor.execute(f"SELECT id, word FROM Words", None)
    for result in cursor:
        for word in all_words:
            if word == result[1]:
                word_id[word] = result[0]
print("Done")

Done


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
import pandas as pd

vec = TfidfVectorizer(min_df=5)
X = vec.fit_transform(content_list.values())

In [7]:
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())
df = df.drop(list(df.filter(regex = "([^\x00-\x7F])+")), axis = 1) # Remove chinese characters
df = df.drop(list(df.filter(regex = "[0-9]+")), axis = 1) # Remove numerical
df = df.drop(list(df.filter(regex = "0x[0-9]+")), axis = 1) # Remove hexadecimal
df

,_comment,abandoning,abide,ability,able,abou,about,aboutcookies,above,abroad,...,youradchoices,yours,yourself,zapier,zealand,zero,zodiac,zone,zones,zuckerberg
0,0.0,0.000000,0.0,0.000000,0.028160,0.0,0.000000,0.0,0.027753,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.000000,0.0,0.000000,0.028160,0.0,0.000000,0.0,0.027753,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.052741,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.018282,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3197,0.0,0.005003,0.0,0.003798,0.014792,0.0,0.072396,0.0,0.014578,0.0,...,0.005003,0.0,0.009363,0.0,0.0,0.0,0.0,0.004709,0.0,0.0
3198,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3199,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3200,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.011536,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [8]:
# for doc, k in zip(df.index.to_list(), content_list.keys()):
#     print(f"Document {doc}, DB id for doc {k}")
#     break
doc_id_dict = {doc: k for doc, k in zip(df.index.to_list(), content_list.keys())}
tuple_list = []
for doc, row in df.iterrows():
    for i, e in enumerate(row):
        try:
            tuple_list += [f"{(doc_id_dict[doc], word_id[row.index[i]], e)}"]
        except Exception:
            continue

In [9]:
data = ','.join(tuple_list)

In [10]:
db("DESCRIBE Words_To_Pages")

('id_Pages', b'int', 'NO', 'PRI', None, '')
('id_Words', b'int', 'NO', 'PRI', None, '')
('tf_idf', b'float', 'NO', '', None, '')


In [12]:
for i in range(len(tuple_list)//1000 +1):
    if(i == len(tuple_list)//1000 +1):
        db("INSERT INTO Words_To_Pages (id_Pages, id_Words, tf_idf) VALUES {0}".format(','.join(tuple_list[i*1000:])))
    else:
        db("INSERT INTO Words_To_Pages (id_Pages, id_Words, tf_idf) VALUES {0}".format(','.join(tuple_list[i*1000:(i+1)*1000])))

db("SELECT * FROM Words_To_Pages LIMIT 10")

(9, 2, 0.10793)
(9, 3, 0.0299577)
(9, 4, 0.00929118)
(9, 5, 0.0528224)
(9, 6, 0.0110006)
(9, 7, 0.0374082)
(9, 9, 0.0281597)
(9, 10, 0.0187202)
(9, 12, 0.0454206)
(9, 13, 0.0195622)


In [19]:
db("SELECT count(id_Words) FROM Words_To_Pages")

(8949590,)


In [17]:
db("SELECT url FROM Pages INNER JOIN Words_To_Pages ON (Words_To_Pages.id_Pages = Pages.id) WHERE Words_To_Pages.id_Words = 178 ORDER BY tf_idf DESC LIMIT 10")

('https://www.python.org/',)
('https://www.python.org/',)
('https://www.python.org/psf/',)
('https://www.python.org/psf',)
('https://www.python.org/psf/',)
('https://www.python.org/psf/',)
('https://www.python.org/psf/sponsors/#microsoft',)
('https://www.python.org/psf/sponsors/#huawei',)
('https://www.python.org/psf/sponsors/#nvidia',)
('https://www.python.org/psf/sponsors/#microsoft',)
